In [1]:
from simulation_loop import SimulationParams, simulate_many, Buffer, simulate_system
from scipy.stats import norm, uniform, gamma, expon
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
np.random.seed(42)

In [2]:
p_ = 0.55
t_ = np.linspace(0, 24, 1000)

class ArrivalDistribution:
    first_round_time = 9
    second_round_time = 15
    def __init__(self, p=0.5, offset=0):
        self.p = p
        self.first_peak = norm(loc=10 + offset, scale=1)
        self.second_peak = norm(loc=15 + offset, scale=1.5)

    def pdf(self, x):
        return self.p * self.first_peak.pdf(x) + (1 - self.p) * self.second_peak.pdf(x)

    def rvs(self, size=()):
        s1 = self.first_peak.rvs(size=size)
        s2 = self.second_peak.rvs(size=size)
        choice = uniform.rvs(size=size) < self.p
        return np.where(choice, s1, s2)

discharge_dist = ArrivalDistribution()
admission_dist = ArrivalDistribution(offset=1)

arrivals = admission_dist.rvs(size=1000)
discharges = discharge_dist.rvs(size=1000)

def service_time_dist():
    service_min_time = 5/60
    return service_min_time + expon(scale=1/60).rvs()


def transport_time_dist(distance_meters, break_down_prob=0):
    if distance_meters == 0:
        return 0
    speed_meters_per_second = 1.4
    transport_time_seconds = gamma(a=distance_meters, scale=1/speed_meters_per_second).rvs()
    time_hours = transport_time_seconds / 3600

    # Add breakdown delay with specified probability
    if break_down_prob > 0 and np.random.rand() < break_down_prob:
        time_hours += 6  # 6 hours of breakdown

    return time_hours

distances = np.array([
    [10, 50, 175, 150],
    [50, 10, 0, 0],
    [175, 0, 10, 0],
    [150, 0, 0, 10],
])

n_elevators = 3
discharge_dist = lambda: ArrivalDistribution().rvs(size=200)

# Sensitivity Analysis - Base case

# Increasing the number of bots

In [ ]:
robot_range = [1, 2, 3, 5, 10]
n_iters = 2

for n_robots in robot_range:
    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=n_robots,
    )

    buffers, *_ = simulate_many(params, n_iters=n_iters)

    # Compute max buffer size per run

    areas = np.percentile(buffers.max(axis=(3)), 99, axis=0) 
    print(areas)

100%|██████████| 2/2 [00:01<00:00,  1.10it/s]


[[89.99 14.   17.97 17.97]
 [ 1.    1.    1.    1.  ]]


100%|██████████| 2/2 [00:01<00:00,  1.19it/s]


[[105.94   5.95   4.98   5.96]
 [  1.     1.99   1.99   1.99]]


100%|██████████| 2/2 [00:01<00:00,  1.20it/s]


[[100.99   1.99   2.     2.  ]
 [  1.     1.99   1.99   1.99]]


100%|██████████| 2/2 [00:01<00:00,  1.18it/s]


[[105.96   1.99   2.99   2.  ]
 [  1.     2.     1.99   1.99]]


100%|██████████| 2/2 [00:01<00:00,  1.06it/s]

[[106.99   1.     1.99   2.98]
 [  1.     2.     1.99   1.99]]


# Changing $\lambda$

In [11]:
lambdas = [0.01, 0.1, 1, 10, 100]  # lambda_arrival values
scales = 1 / np.array(lambdas)
n_iters = 5  # number of simulation runs

for lam in scales:
    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=1,
    )

    # Run simulations
    buffers, *_ = simulate_many(params, n_iters=n_iters, lambda_arrival=lam)

    areas = np.percentile(buffers.max(axis=(3)), 99, axis=0) 
    print(areas)

100%|██████████| 5/5 [00:04<00:00,  1.15it/s]


[[97.96 14.   15.92 16.96]
 [ 1.   46.92 46.92 46.96]]


100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


[[94.96 17.92 17.96 17.96]
 [ 1.   11.96 11.96 12.  ]]


100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


[[99.76 18.76 19.84 18.96]
 [ 1.    2.    2.96  2.  ]]


100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


[[93.88 14.96 15.96 16.88]
 [ 1.    1.    1.    1.  ]]


100%|██████████| 5/5 [00:04<00:00,  1.09it/s]

[[96.8  13.   17.   18.96]
 [ 1.    0.96  1.    1.  ]]


# Random robot breakdowns

In [ ]:
breakdown_probs = [0, 1/100, 2/100, 5/100]

n_iters = 5  # Number of simulation runs per case


for prob in breakdown_probs:

    def transport_time_dist(distance_meters):
        if distance_meters == 0:
            return 0
        speed_mps = 1.4
        transport_time = gamma(a=distance_meters, scale=1 / speed_mps).rvs()
        time_hours = transport_time / 3600
        if np.random.rand() < prob:
            time_hours += 6  # Add 6 hours if breakdown occurs
        return time_hours

    params = SimulationParams(
        discharge_dist,
        service_time_dist,
        transport_time_dist,
        distances,
        arrival_weights=np.ones(n_elevators) / n_elevators,
        discharge_weights=np.ones(n_elevators) / n_elevators,
        n_elevators=n_elevators,
        n_robots=1,
    )

    # Run simulations
    buffers, *_ = simulate_many(params, n_iters=n_iters)

    areas = np.percentile(buffers.max(axis=(3)), 99, axis=0) 
    print(areas)

100%|██████████| 5/5 [00:04<00:00,  1.16it/s]


[[98.8  16.   18.76 15.92]
 [ 1.    2.    2.    2.96]]


100%|██████████| 5/5 [00:04<00:00,  1.25it/s]


[[45.56 67.88 63.88 66.84]
 [46.48  1.    1.    1.  ]]


100%|██████████| 5/5 [00:04<00:00,  1.25it/s]


[[19.68 67.8  76.48 64.96]
 [22.6   1.96  1.96  1.96]]


100%|██████████| 5/5 [00:04<00:00,  1.19it/s]

[[ 8.92 68.88 82.84 68.8 ]
 [11.88  1.96  1.96  1.96]]
